# Neural Networks Lab
## Preprocessing Data
For this lab, there isn't any preprocessing! However, importing the data is pretty tricky, as it involves dealing with binaries and libraries and whatnot. I used [this Stack Overflow thread](https://stackoverflow.com/questions/48355140) to help import the data.

In the cell below, I've imported the training and validation datasets for you in advance. Simply run it, and continue with the rest of the lab!

In [ ]:
from io import BytesIO
import numpy as np
import requests

# Load and read the data - https://stackoverflow.com/questions/48355140
data = np.load(
    BytesIO(
        requests.get(
            'https://github.com/Endothermic-Dragon/Polygence/raw/master/Jupyter%20Notebooks/Neural%20Networks/mnist.npz',
            stream = True
        ).raw.read()
    )
)

# Assign the data to appropriate variables
trainingData = data["x_train"]
trainingOutputs = data["y_train"]
validationData = data["x_test"]
validationOutputs = data["y_test"]

# Delete unnecessary variables
del BytesIO, requests, data

print("Successfully loaded dataset!")

## Visualize the Data

Next, let's visualize the data you're working with. All of the training data is in the form of a $28 \times 28$ array, with each cell holding a pixel value from 1 to 255. The code below displays an $n \times n$ grid of the training data. Analyze how the code works and update the value of $n$ to visualize a larger sample.

In [ ]:
from matplotlib import pyplot as plt
n=5
fig, axes = plt.subplots(nrows=n, ncols=n, figsize=(n*1.5,n*1.5))

for i in range(n**2):
    ax = axes[i//n][i%n]
    ax.axis("off")
    ax.imshow(trainingData[i], cmap=plt.get_cmap('gray'))

fig.subplots_adjust(wspace=0.2)
plt.show()
del n, fig, axes, ax

Currently, each data point in the training and validation dataset inputs are in a 28 by 28 array. "Flatten" these data points so that each input is simply a series of 784 pixel values.

You may find the `.reshape(...)` function helpful (documentation [here](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html)), along with the `.shape` property of an array, which returns the length of the dimensions (documentation [here](https://numpy.org/doc/stable/reference/generated/numpy.shape.html)).

To more easily understand the `shape` attribute, it might be easier to know how it works on the "inside" rather than to visualize it in 2D or 3D. Getting the `shape` attribute returns an array. The first element of the array specifies how many lists/elements are in the overarching array. The second element of the array specifies how many lists/elements are in each list of the overarching array. The third element of the list specifies how many lists/elements are in each list in each list of the overarching array. When visualized in 2D, the first number specifies the number of in the array, which is just the number of rows; the second number specifies the number of elements in each list of the array, which is the number of columns.

Note that when reshaping, your goal is to preserve the number of rows as that corresponds to individual data points, but to "merge" the 2D arrays in each row into a 1D array (via the use or `.reshape(...)`).

<details>
<summary>Stuck or completed? Click here to reveal a working example program that you could've written.</summary>

```python
# Reshape the training data
trainingData = trainingData.reshape(trainingData.shape[0], trainingData.shape[1]*trainingData.shape[2])
# Reshape the validation data
validationData = validationData.reshape(validationData.shape[0], validationData.shape[1]*validationData.shape[2])
```
</details>

The training and validation outputs are simply in the form of an integer between 0 and 9 (inclusive) as of now. However, in order to effectively calculate cost functions and gradients, it's important to have them in the format of an array.

Let's take an example. For a row, let's assume that the expected output is 2. Since we are dealing with simulated neurons, we want the element at index `2` to be 1, and all the other 9 possible outputs to be 0. Thus, the corresponding array to replace this output would be `[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]`.

Your job is to convert `trainingOutputs` and `validationOutputs` to the required form in the next cell. Note that this process doesn't necessarily have to be efficient, since it's only done once.

<details>
<summary>What I did</summary>

I initialized a new 2D array with the same number of rows and 10 columns filled with zeros using `np.zeros(...)`. After that, I looped through each row in `trainingOutputs`, and updated the index in the newly initialized array with 1's at the appropriate location. Then, I set `trainingOutputs` equal to the new array I created.

Finally, I repeated this process for `validationOutputs`.

</details>

After completing this step, delete any unnecessary variables using `del [first variable name], [second variable name], ...`

<details>
<summary>Stuck or completed? Click here to reveal a working example program that you could've written.</summary>

```python
# Initialize a new array of 0s
newTrainingOutputs = np.zeros((trainingOutputs.shape[0], 10))

# Loop through each row of training outputs
for i in range(len(trainingOutputs)):
    # Set the appropriate index to 1
    newTrainingOutputs[i, trainingOutputs[i]] = 1

# Replace old array and delete temporary array
trainingOutputs = newTrainingOutputs
del newTrainingOutputs

# Initialize a new array of 0s
newValidationOutputs = np.zeros((validationOutputs.shape[0], 10))

# Loop through each row of validation outputs
for i in range(len(validationOutputs)):
    # Set the appropriate index to 1
    newValidationOutputs[i, validationOutputs[i]] = 1

# Replace old array and delete temporary array
validationOutputs = newValidationOutputs
del newValidationOutputs
```

</details>

## Training
### Quick Note
This part of the lab is quite long, despite the decievingly short appearance. If this takes two or three days, don't worry! That's expected. In addition, if you get stuck, there are also individual solutions below (as always)!

In addition, you might have to some juggling, as Google Colab has a limited amount of RAM to work with. You can do this by deleting larger arrays or variables that you don't need anymore using `del [first variable name], [second variable name], ...`. Or, if you have python installed, you can download this colab and do the lab locally (no need to worry about RAM).

### Neural Network Structure
While a neural net's structure is completely up to you to decide, we'll be using 25 nodes for the hidden layer (excluding bias term) and 10 nodes for the output layer.

Our images have dimensions of 28 by 28 pixels - in total, that's 784 pixels. Given that we also need a bias term, we can say we have 785 input features.

Now, we have the dimensions of our weights by layer, along with our forward propagation procedure:
- First to second layer - 785 by 25 array
    - Note that currently we have 784 features - we need to add another column at the beginning filled with 1s, which will serve as the bias node.
    - We'll by multiplying `trainingData` with this matrix, which should return a matrix with the same numer of rows and 25 columns. Then, we'll apply a sigmoid to each element.
- Second to third layer - 26 by 10 array
    - After getting the output from the last layer, we'll add another column of 1s for the bias node in the second layer.
    - Next, we once again multiply and apply a sigmoid, to obtain a matrix with 10 columns, one for each node in the output layer.

Great, now we know exactly what we have to do to go forward! However, this is easier said than done. Let's break the big concept down to individual steps.

### Initializing Weights
When making a neural network, we first have to initialize the weights at random. But where exactly do we choose this "random" from? A good choice comes from the range between $-\epsilon$ and $\epsilon$, where $\epsilon$ comes from the following formula:

<h3>

$$\epsilon = \sqrt{\frac{6}{\text{\# of input nodes} + \text{\# of nodes in first hidden layer}}}$$

</h3>

In the cell below, calculate $\epsilon$ and use `np.random.uniform(...)` to make 1D arrays of the appropriate sizes, sampled from the appropriate range. You can access the documentation [here](https://numpy.org/doc/stable/reference/random/generated/numpy.random.uniform.html).

### Basic Functions
First of all, write the function for `sigmoid`, keeping in mind that it should be able to handle both numpy arrays and individual numbers. Once again, the formula is:

<h3>

$$\sigma(n) = \frac{1}{1+e^{-n}}$$

</h3>

Using the explanation above, write the functions `predict`, `J`, and `forwardProp`. You may find [`np.hstack(...)`](https://numpy.org/doc/stable/reference/generated/numpy.hstack.html), [`np.ones(...)`](https://numpy.org/doc/stable/reference/generated/numpy.ones.html), and dot product [`@`](https://numpy.org/doc/stable/reference/generated/numpy.dot.html) functions useful.

Here are the specifications of their inputs and outputs:
- `predict`
    - The input is simply the array `trainingData`
    - Stack a column of 1's before it via `np.hstack(...)`
    - Multiply with first layer of weights
    - Apply a sigmoid
    - Stack a column of 1's before it again
    - Multiply with second layer of weights
    - Apply a sigmoid
- `J`
    - Pass through the `predict` function
    - Apply the loss function without the negative (we'll apply that at the end)
    - Take the sum of each row and the average of each column (can be done in either order)
    - Negate the value
    - Apply regularization
        - Due to how dot products work in matrices, the topmost row in each array of weights is multiplied to the bias node. Remove this row before squaring, applying `np.sum(...)`, and multiplying by $\lambda$ (passed into function as `l`). View [this documentation](https://numpy.org/doc/stable/user/basics.indexing.html) for a refresher on how to work with indices.
- `forwardProp`
    - Perform the calculations as `predict`
    - Return a python list of "outputs" from each layer (should return a list of 3 numpy arrays)
        - Make sure to apply `sigmoid` and add a bias column
        - For first layer, since it's just the input data, you don't need to apply a `sigmoid`
    - The return values will be used in back propagation when calculating the gradients of the weights

In [ ]:
epsilon = 0
thetas1 = []
thetas2 = []
weights = np.array([thetas1, thetas2], dtype=object)
del epsilon, thetas1, thetas2

def sigmoid(n):
    return

def predict(weights, data=trainingData):
    return

def J(weights, l, data=trainingData, outputs=trainingOutputs):
    return

def forwardProp(weights, data=trainingData):
    return

<details>
<summary>Stuck or completed? Click here to reveal a possible solution to initializing the weights.</summary>

```python
epsilon = np.sqrt(6/(784+25))
thetas1 = np.random.uniform(-epsilon, epsilon, (785, 25))
thetas2 = np.random.uniform(-epsilon, epsilon, (26, 10))
weights = np.array([thetas1, thetas2], dtype=object)
del epsilon, thetas1, thetas2
```
</details>

<details>
<summary>Stuck or completed? Click here to reveal a possible solution for the <code>sigmoid</code> function.</summary>

```python
def sigmoid(n):
    return 1/(1+np.e**(-n))
```
</details>

<details>
<summary>Stuck or completed? Click here to reveal a possible solution for the <code>predict</code> function.</summary>

```python
def predict(weights, data=trainingData):
    # Apply sigmoid to each node after propagating from first to second layer
    layer2_pre = sigmoid(
        # Append column of 1s to beginning
        np.hstack([
            np.ones((data.shape[0], 1)),
            data
        ])

        # Multiply by first set of weights and add up at each node
        @ weights[0]
    )

    # Apply sigmoid to each node after propagating from second to third layer
    # Return output after sigmoid
    return sigmoid(
        # Append column of 1s to beginning
        np.hstack([
            np.ones((layer2_pre.shape[0], 1)),
            layer2_pre
        ])

        # Multiply by second set of weights and add up at each node
        @ weights[1]
    )
```
</details>

<details>
<summary>Stuck or completed? Click here to reveal a possible solution for the <code>J</code> function.</summary>

```python
def J(weights, l, data=trainingData, outputs=trainingOutputs):
    # Plug into formula
    result = -np.sum(np.mean(
        outputs*np.log(predict(weights, data))
        + (1 - outputs)*np.log(1 - predict(weights, data))
    , 0))

    # Add regularization terms from first set of weights (excluding bias term)
    result += l*np.sum(weights[0][1:]**2)
    # Add regularization terms from second set of weights (excluding bias term)
    result += l*np.sum(weights[1][1:]**2)

    # Return cost
    return result
```
</details>

<details>
<summary>Stuck or completed? Click here to reveal a possible solution for the <code>forwardProp</code> function.</summary>

```python
def forwardProp(weights, data=trainingData):
    # Add bias term to features and save it
    layer1 = np.hstack([
        np.ones((data.shape[0], 1)),
        data
    ])

    # Propagate one layer and apply sigmoid
    layer2_pre = sigmoid(layer1 @ weights[0])

    # Add bias term to second layer and save it
    layer2 = np.hstack([
        np.ones((layer2_pre.shape[0], 1)),
        layer2_pre
    ])

    # Propagate to output layer
    layer3 = sigmoid(layer2 @ weights[1])

    # Return outputs of each layer
    return [layer1, layer2, layer3]
```
</details>

### Final Boss - Backpropagation
This part of the lab takes the most time. If you're not prepared to spend, at the very least, half an hour on understanding this, then I'd recommend simply copying and pasting the code from the solution below.

That being said, this is the most fun part to understand. After all, who doesn't want to be able to brag that they know and are able to apply advanced PhD level material from the late 1900s? While it may take a bit of time, the euphoria felt when your code works is immeasurable.

Before we begin, let's recap the algorithm, which is repeated for each row of data:
- Calculate the error for the last layer. Assign this to `delta2`.
- Multiply each error by the weight at each connection, add up products at the second layer. Undo the sigmoid. Assign this to `delta1`.
- No need to calculate error for the first layer
- For each connecting weight, multiply the output value of the previous layer's connected node to the error value of the next layer's connected node
- Repeat for all weights and for all rows in the training data

Now, since we're trying to optimize our RAM, here's an outline of how to do this algorithm successfully and efficiently:
- Get forward propagation results
- Delete `data` as you don't need it anymore
    - You also have a similar copy from your forward propagation outputs
- Calculate `delta2`, and then delete any variables you may have created to calculate it
- Delete `outputs` as you don't need it anymore
- Calculate the derivative of the second array of weights, assign it to `deriv_weights_1`
    - Account for the regularization term
        - Account for the fact that first row isn't regularized
- Calculate `delta1`, and then delete any variables you may have created to calculate it
- Delete `delta2` as you don't need it anymore
- Calculate the derivative of the first layer of weights, assign it to `deriv_weights_0`
    - Calculate the derivative from the unregularized cost function
    - Delete `forwardPropData` and `delta1` as you don't need them anymore
    - Account for the regularization term
        - Account for the fact that first row isn't regularized
    - Put `deriv_weights_0` and `deriv_weights_1` in a ragged numpy array

In addition to this procedure, you may need a few extra pieces of information.
- Calculating `delta2`
    - This is actually quite similar to how the gradients are calculated logistic regression lab. First, you find the derivative of the cost function with respect to each node in the prediction function output and undo the sigmoid for each node. Note how you do NOT take the sum or mean. Here's the formula, which you repeat for all 60,000 rows (i) and 10 columns (j):

<h3>

$$\left(\frac{1-y_{i,j}}{1-\sigma(\text{node sum}_{i,j})} -\frac{y_{i,j}}{\sigma(\text{node sum}_{i,j})} \right) \cdot \sigma(\text{node sum}_{i,j})(1 - \sigma(\text{node sum}_{i,j}))$$

</h3>

    - Note that we don't need the node sum to calculate this, as:

<h3>

$$\sigma(\text{node sum}_{i,j}) = \text{node output}_{i,j}$$

</h3>

- Calculating `delta1`
    - Each row in `weights[1]` represents all the connections from one node in the previous layer to all the nodes in the next layer. Consequently, in its transposition, each column represents all the connections from all the nodes in the next layer to a single node in the previous layer.
    - Performing a dot multiplication between `delta2` and the transposition of `weights[1]` not only multiplies each error to the right weight, but also adds them up. You're almost done with the calculation for `delta1` - you just have to undo the sigmoid! You can do this by multiplying by:

<h3>

$$\text{second layer node output}_{i,j} \cdot (1 - \text{second layer node output}_{i,j})$$

</h3>

    - After this step, remember to strip the first column because the errors for the bias term won't help us - not to mention that the entire column is 0 anyways, as a result of undoing the sigmoid.
- Calculating `deriv_weights_0` and `deriv_weights_1` WITHOUT regularization
    - At this point, you're trying to somehow combine two large 2D arrays, but you don't know how. To simplify this case, let's just take the first row from each. You know that the output should be of the same shape as the corresponding layer of weights. In addition, you know that the number of elements in the row of errors corresponds to the number of columns in the corresponding layer's weight. Similarly, the number of elements in the row of the previous layer's outputs corresponds to the number of rows in the corresponding layer's weight.
    - Now, you can leverage the power of [broadcasting (see figure 4)](https://numpy.org/doc/stable/user/basics.broadcasting.html)! Specifically, you can make the row of the previous layer's outputs into a vertical 2D array and the row of errors into a horizontal 2D array, and multiply them.
    - You need to repeat this concept, but on a 3D scale. How do you do this, you ask? Well, you can turn each 2D array into a 3D array, using `np.newaxis` (see [Stack Overflow](https://stackoverflow.com/questions/29241056) and [numpy documentation](https://numpy.org/doc/stable/reference/constants.html#numpy.newaxis))! Before doing anything, take note that you want to conserve the number of rows.
    - In a 2D array `a`, `a[:, np.newaxis, :]` will add brackets one layer in. Specifically, each row will be double-braced with brackets.
        - Essentially, each row will have a horizontal 2D array in it.
    - In a 2D array `a`, `a[:, :, np.newaxis]` will add brackets two layers in. Specifically, each element will have a brace around it.
        - Essentially, each row will have a vertical 2D array in it.
    - Ok, now you have a huge 3D array! Time to collapse it. Take the mean of all the 2D matrices in the 60,000 row array. Remember to set the axis parameter properly.
- Adding regularization to `deriv_weights_0` and `deriv_weights_1`
    - The derivative of the regularization term is $2\lambda\theta$. For each layer, calculate this value for the entire array of $\theta$s (remember that `l` represents $\lambda$).
    - Before adding this to `deriv_weights_0` or `deriv_weights_1`, remember that the bias term is supposed to be excluded. Accordingly, set the first row of the array of weights to 0. [Here's a shortcut you may find helpful](https://stackoverflow.com/questions/17482955).
    - Add it to the nonregularized weight derivatives!
- To transpose an array `a`, simply use `a.T`.
- To create a ragged numpy array, use `np.array(..., dtype=object)`. This is the same way the ragged array for `weights` is created.

Whew! That was a LOT. This is one of the two main reasons people use libraries to do this for them. The other reason is that neural networks are very expensive to train, both in RAM and in time.

Good luck on the implementation, and if you need any help, know that you can always refer to the solution snippets below!

In [ ]:
def backProp(weights, l, data=trainingData, outputs=trainingOutputs):
    return

<details>
<summary>Stuck or completed? Click here to reveal a possible solution for calculating <code>delta2</code>.</summary>

```python
# Get output from forward propagation
forwardPropData = forwardProp(weights, data)
# Isolate last layer in seperate variable for readability and aesthetics
preds = forwardPropData[2]
# Calculate the errors on the last layer
delta2 = ((1-outputs)/(1-preds) - outputs/preds)*preds*(1-preds)
```
</details>

<details>
<summary>Stuck or completed? Click here to reveal a possible solution for the <code>backProp</code> function.</summary>

```python
def backProp(weights, l, data=trainingData, outputs=trainingOutputs):
    # Get output from forward propagation
    forwardPropData = forwardProp(weights, data)
    # Delete unnecessary variables
    del data

    # Isolate last layer in seperate variable for readability and aesthetics
    preds = forwardPropData[2]
    # Calculate the errors on the last layer
    delta2 = ((1-outputs)/(1-preds) - outputs/preds)*preds*(1-preds)
    # Delete unnecessary variables
    del outputs, preds

    # Get the derivative of the second set of weights
    # Unregularized
    deriv_weights_1 = np.mean(
        delta2[:,np.newaxis,:]*forwardPropData[1][:,:,np.newaxis]
    , 0)
    # Add regularization
    reg_term = 2*l*weights[1]
    # Zero out bias term regularizations
    reg_term[0] = 0
    # Put it together
    deriv_weights_1 += reg_term
    # Delete unnecessary variables
    del reg_term

    # Calculate the errors on the hidden second layer + remove first column
    delta1 = (
        delta2 @ weights[1].T * forwardPropData[1]*(1-forwardPropData[1])
    )[:,1:]
    # Delete unnecessary variables
    del delta2

    # Get the derivative of the first set of weights
    # Unregularized
    deriv_weights_0 = np.mean(
        delta1[:,np.newaxis,:]*forwardPropData[0][:,:,np.newaxis]
    , 0)
    # Delete unnecessary variables
    del forwardPropData, delta1
    # Add regularization
    reg_term = 2*l*weights[0]
    # Zero out bias term regularizations
    reg_term[0] = 0
    # Put it together
    deriv_weights_0 += reg_term

    # Group derivatives into the same format as the weights and return
    return np.array([deriv_weights_0, deriv_weights_1], dtype=object)
```
</details>

### Learning Rate
Taking inspiration from [this article](https://towardsdatascience.com/https-medium-com-super-convergence-very-fast-training-of-neural-networks-using-large-learning-rates-decb689b9eb0) and the concept of superconvergence, I've implemented a variable learning rate below. Specifically, the inpiration comes from the typical trend seen when training. Initially, there are steep gradients that you can traverse quickly, but then you reach flat valleys. In those areas, a higher learning rate helps speed things up. Then, as you approach a local minimum, you slow down again to converge carefully.

After this intuition, I tinkered around with the learning rate for some more time, along with the regularization coefficient to pick good settings for both. Speaking of the regularization coefficient...

### Regularization
I also did quite a bit of testing with coefficients! I used the variable learning rate from the generator function below for all of these tests, and ran them for 100 iterations. Here's a summary of my results:
- Coefficient of `0`: average of 81.73% accuracy
    - 80.68%
    - 78.03%
    - 84.04%
    - 84.17%
- Coefficient of `0.0005`: average of 82.245% accuracy
    - 82.63%
    - 79.03%
    - 85.71%
    - 81.61%
- Coefficient of `0.001`: average of 84.1525% accuracy
    - 83.72%
    - 84.29%
    - 84.47%
    - 84.13%
- Coefficient of `0.0015`: average of 83.2325% accuracy
    - 82.96%
    - 86.14%
    - 81.74%
    - 82.09%
- Coefficient of `0.002`: average of 83.7975% accuracy
    - 84.67%
    - 84.09%
    - 81.57%
    - 84.86%
- Coefficient of `0.0025`: average of 83.405% accuracy
    - 81.08%
    - 85.73%
- Coefficient of `0.003`: average of 82.005% accuracy
    - 81.81%
    - 82.2%

To me, both `0.001` and `0.0015` seemed like good candidates, so I ran both of them for 300 iterations. I got 87.52% accuracy for `0.001`, and 87.34% accuracy for `0.0015`.

To conclude the results of this testing, I recommend using `0.001` as your learning coefficient, and the variable learning rate generator below to speed up your training process and achieve a high accuracy.

Update the code below to use the appropriate value for $\lambda$ (`l`), and feel free to adjust the number of iterations run.

In [ ]:
def stepLearner():
    yield from np.linspace(0.2, 0.5, 20)
    yield from np.linspace(0.4, 0.25, 15)
    yield from np.linspace(0.25, 0.2, 25)
    for _ in range(6):
        yield from np.linspace(0.2, 0.15, 10)
    for _ in range(5):
        yield from np.linspace(0.15, 0.1, 10)
    for _ in range(5):
        yield from np.linspace(0.1, 0.07, 10)
    while True:
        yield 0.05

# Update value of lambda
l = 0
learningRate = stepLearner()
iterations = 100
cost_history = [J(weights, l)]

for i in range(iterations):
    weightGradients = backProp(weights, l)
    weights = weights - next(learningRate) * weightGradients
    del weightGradients
    cost_history.append(J(weights, l))
    print(f"Iteration {i+1}, Cost {cost_history[-1]}")

print("Initial cost:", cost_history[0])
print("Final cost:", cost_history[-1])
plt.plot(np.arange(iterations+1), cost_history)
plt.show()

## Judging Model
After all of that training, it's time to judge how good our model is.

First, we need to check overfitting. Calculate the cost of the model on the validation dataset. Make sure to pass the value of $\lambda$. Check if it's comparable with the cost from our training data.

<details>
<summary>Stuck or completed? Click here to reveal a working example program that you could've written.</summary>

```python
print("Validation cost:", J(weights, l, validationData, validationOutputs))
```
</details>

Now, to find the accuracy, calculate what percent of your model's predictions on the validation dataset is correct. You may find the `np.argmax(...)` function helpful. Also take note that you can call `np.sum(...)` on a boolean list, which will treat `True` values as 1 and `False` values as 0.

Your accuracy should be around 84% for 100 iterations. For more iterations, you could achieve accuracy values closer to 90%.

<details>
<summary>Stuck or completed? Click here to reveal a working example program that you could've written.</summary>

```python
comparison = np.argmax(predict(weights, validationData), 1) == np.argmax(validationOutputs, 1)
print(f"Model accuracy: {(np.sum(comparison) / comparison.size * 100).round(2)}%")
```
</details>

## Credits
- This lab used the MNIST handwritten digits dataset. You can find the original dataset [here](https://yann.lecun.com/exdb/mnist/).